In [1]:
!pip3 install ACSConv

     |████████████████████████████████| 88 kB 666 kB/s eta 0:00:01
     |████████████████████████████████| 125 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 5.4 MB/s eta 0:00:01
  Created wheel for ACSConv: filename=ACSConv-0.1.1-py3-none-any.whl size=24202 sha256=3be30739b7fce33b82ab0e8a4b8bd54210b746bc9d57bb41efa2e36da44d526f
  Stored in directory: /home/user1/.cache/pip/wheels/46/0a/29/16aa677f167ed02fb8eb81ffef716f426304fe749db6d05135
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=7cef1badddfe829769318035c9d086595f924ffe6e57a071aa42e97a9c3dadaf
  Stored in directory: /home/user1/.cache/pip/wheels/5b/eb/43/7295e71293b218ddfd627f935229bf54af9018add7fbb5aac6
Successfully built ACSConv fire
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import PIL
from PIL import Image
import torchvision
import torch
import glob
import nibabel as nib
import time
from sklearn.model_selection import StratifiedKFold

import torch.nn.functional as F
from torchvision import utils, transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import KFold
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
import intel_extension_for_pytorch as ipex

myseed = 1
torch.manual_seed(myseed)
np.random.seed(myseed)

num_classes=2

%matplotlib inline

In [2]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cpu


### 1) Data wrangling

In [3]:
#!ls a2/ADNI2_ALL_T1

#### Retrieve img filenames and paths

In [34]:
# specifying the zip file name
adni_num = "2"
data_path = f"a{adni_num}"
#img_path = "a2/ADNI2_ALL_T1/"

img_df_filename=f"adni{adni_num}_paths.pkl"
filename=os.path.join(data_path, img_df_filename)
img_df=pd.read_pickle(filename)  

print(f"Final data has {len(img_df)}")
img_df.head()

Final data has 260


,PTID,IMG_PATH,SRC_PATH
0,094_S_4459,a2/ADNI2_ALL_T1/094_S_4459.nii.gz,a2/T1/094_S_4459/Accelerated_SAG_IR-SPGR/fsl_a...
1,137_S_4211,a2/ADNI2_ALL_T1/137_S_4211.nii.gz,a2/T1/137_S_4211/MPRAGE/fsl_anat.anat/T1_to_MN...
2,052_S_4959,a2/ADNI2_ALL_T1/052_S_4959.nii.gz,a2/T1/052_S_4959/Accelerated_Sag_IR-SPGR/fsl_a...
3,022_S_4196,a2/ADNI2_ALL_T1/022_S_4196.nii.gz,a2/T1/022_S_4196/MPRAGE/fsl_anat.anat/T1_to_MN...
4,029_S_4290,a2/ADNI2_ALL_T1/029_S_4290.nii.gz,a2/T1/029_S_4290/Accelerated_SAG_IR-FSPGR/fsl_...


#### Retrieve ADNI table, with normalized values (ADNI_ready.csv)

In [39]:
data_path = "ADNI_csv/"
filename = "ADNI_ready.csv"

adni_tabular = pd.read_csv(os.path.join(data_path, filename))
adni_tabular.head()


# Print 
print(f"ALL adni has {len(adni_tabular)} entries")
print(f"Class distribution is organized as follow:")
print(f"\n {adni_tabular['labels'].value_counts()}")

adni_tabular=adni_tabular[adni_tabular['SRC']==f"ADNI{adni_num}"]

print(f"Adni{adni_num} has {len(adni_tabular)} entries")
print(f"Class distribution is organized as follow:")
print(f"Final:\n {adni_tabular['labels'].value_counts()}")


## Check for duplicated rows
dup = adni_tabular[adni_tabular.duplicated()]
if not dup.empty:
    print(f"WARNING: Dataframe contains duplicated rows!!!")

ALL adni has 961 entries
Class distribution is organized as follow:

 0    583
1    378
Name: labels, dtype: int64
Adni2 has 288 entries
Class distribution is organized as follow:
Final:
 0    145
1    143
Name: labels, dtype: int64


#### Merge the images and tabular dataframes

In [46]:
# Extract only ADNI2 subjects
adni = pd.merge( left=img_df, right=adni_tabular, how="inner", on="PTID", 
                      suffixes=("_x", "_y"),copy=False, indicator=False, validate="one_to_one")

In [50]:
print(f"Class distribution is organized as follow:")
print(f"Final:\n {adni['labels'].value_counts()}")
adni.head()

Class distribution is organized as follow:
Final:
 0    145
1     88
Name: labels, dtype: int64


,PTID,IMG_PATH,SRC_PATH,Unnamed: 0,labels,AGE,PTGENDER,ADAS11,MMSE,FAQ,CDRSB,APOE4,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,SRC
0,137_S_4211,a2/ADNI2_ALL_T1/137_S_4211.nii.gz,a2/T1/137_S_4211/MPRAGE/fsl_anat.anat/T1_to_MN...,277,1,1.010212,0.0,-0.049563,-1.411360,-0.317334,1.313287,-0.793893,-1.144230,-0.772101,-1.218351,ADNI2
1,052_S_4959,a2/ADNI2_ALL_T1/052_S_4959.nii.gz,a2/T1/052_S_4959/Accelerated_Sag_IR-SPGR/fsl_a...,147,1,0.509276,1.0,0.550948,-0.505942,1.973669,1.313287,0.722869,-0.720327,-1.118016,0.396038,ADNI2
2,022_S_4196,a2/ADNI2_ALL_T1/022_S_4196.nii.gz,a2/T1/022_S_4196/MPRAGE/fsl_anat.anat/T1_to_MN...,81,0,0.641877,1.0,-1.250585,1.003090,-0.699168,-0.756357,-0.793893,0.904636,1.303392,-0.814754,ADNI2
3,029_S_4290,a2/ADNI2_ALL_T1/029_S_4290.nii.gz,a2/T1/029_S_4290/Accelerated_SAG_IR-FSPGR/fsl_...,99,0,-0.080060,1.0,0.310743,0.701283,-0.699168,-0.756357,0.722869,0.198130,-0.080270,0.396038,ADNI2
4,127_S_4940,a2/ADNI2_ALL_T1/127_S_4940.nii.gz,a2/T1/029_S_4290/Accelerated_SAG_IR-FSPGR/fsl_...,230,1,2.247818,1.0,2.112276,-0.807748,0.064500,0.278465,-0.793893,-1.497483,-0.426185,-0.007560,ADNI2


In [55]:
## Check for subjects with missing images/tabular

images_pk = img_df['PTID'].tolist()
adni_pk = adni_tabular['PTID'].tolist()

intersection = list(set(images_pk) - set(adni_pk))
print(intersection)
for ptid in intersection:
    
    if ptid in images_pk:
        print(f"{ptid} Missing tabular")

    elif ptid in adni_pk:
        print(f"{ptid} Missing Image")
    

['109_S_4378', '003_S_4839', '136_S_4726', '013_S_4731', '036_S_4740', '035_S_4783', '082_S_5029', '127_S_4749', '041_S_4083', '098_S_4002', '032_S_4304', '094_S_4460', '057_S_4110', '037_S_4001', '123_S_4362', '036_S_5149', '068_S_5206', '116_S_4537', '094_S_4459', '099_S_4124', '003_S_4840', '136_S_4727', '012_S_4642', '100_S_5246', '041_S_4509', '068_S_5146', '100_S_4511']
109_S_4378 Missing tabular
003_S_4839 Missing tabular
136_S_4726 Missing tabular
013_S_4731 Missing tabular
036_S_4740 Missing tabular
035_S_4783 Missing tabular
082_S_5029 Missing tabular
127_S_4749 Missing tabular
041_S_4083 Missing tabular
098_S_4002 Missing tabular
032_S_4304 Missing tabular
094_S_4460 Missing tabular
057_S_4110 Missing tabular
037_S_4001 Missing tabular
123_S_4362 Missing tabular
036_S_5149 Missing tabular
068_S_5206 Missing tabular
116_S_4537 Missing tabular
094_S_4459 Missing tabular
099_S_4124 Missing tabular
003_S_4840 Missing tabular
136_S_4727 Missing tabular
012_S_4642 Missing tabular


### 2) Dataset creator

In [89]:
class ImgDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, adni_df):
        self.adni = adni_df
        
    def __len__(self):
        return len(self.adni)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        line = self.adni.iloc[idx, 0:]
        # Get Label
        y = line['labels']

        # Get tabular
        tabular = line[['AGE','PTGENDER','ADAS11', 'MMSE', 'FAQ', \
                   'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting', \
                   'CDRSB', 'APOE4']]
        tabular = torch.DoubleTensor(tabular)
        
        # Get image
        image = nib.load(line['IMG_PATH'])
        image = image.get_fdata() 
        #image = image[..., :3]
        image = transforms.functional.to_tensor(image)
        image = image.unsqueeze(dim=0)
        
        return image, tabular, y

img_data = ImgDataset(adni_df=adni)

### Detach test set and use remaining data for train-val k-fold split (further down)

In [90]:
from sklearn.model_selection import train_test_split

labels = adni['labels'].tolist()
# Split data into train+val and test set indexes
tv_idx, test_idx = train_test_split(np.arange(len(labels)), test_size=0.1,shuffle=True,stratify=labels)

# Create train+val dataframe and show class balance
adni_tv = adni.iloc[tv_idx]
print(adni_tv.groupby(["labels"]).count())
tv_data = ImgDataset(adni_df=adni_tv)

# Create test dataframe and show class balance
adni_test = adni.iloc[test_idx]
print(adni_test.groupby(["labels"]).count())
test_data = ImgDataset(adni_df=adni_test)

        PTID  IMG_PATH  SRC_PATH  Unnamed: 0  AGE  PTGENDER  ADAS11  MMSE  \
labels                                                                      
0        130       130       130         130  130       130     130   130   
1         79        79        79          79   79        79      79    79   

        FAQ  CDRSB  APOE4  RAVLT_immediate  RAVLT_learning  RAVLT_forgetting  \
labels                                                                         
0       130    130    130              130             130               130   
1        79     79     79               79              79                79   

        SRC  
labels       
0       130  
1        79  
        PTID  IMG_PATH  SRC_PATH  Unnamed: 0  AGE  PTGENDER  ADAS11  MMSE  \
labels                                                                      
0         15        15        15          15   15        15      15    15   
1          9         9         9           9    9         9       9     9   

     

In [91]:
i = 14
print(f"{test_data[i][0].size()}, tabular = {test_data[i][1]}, label = {test_data[i][2]}")

len(test_data[i][1])

torch.Size([1, 91, 91, 109]), tabular = tensor([ 1.2165,  0.0000, -0.4099,  1.0031, -0.6992, -0.0138,  0.6116, -0.4112,
        -0.7564,  0.7229], dtype=torch.float64), label = 0


10

### 3. Model: img+tabular

In [92]:
def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, block_inplanes, \
                 n_input_channels=3, conv1_t_size=7, \
                 conv1_t_stride=1, no_max_pool=False, \
                 shortcut_type='B', widen_factor=1.0, \
                 n_classes=400, img_contribution=10, \
                 tabular_val=10, tabular_contribution=10):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, img_contribution)
        
        #qui ho cambiato da num_classes a 10, per far cat con testo
        #TESTO
        self.relu = nn.ReLU()
        self.ln1 = nn.Linear(tabular_val, 50) #23 sono le colonne in input
        self.ln2 = nn.Linear(50, 50)
        self.ln3 = nn.Linear(50, tabular_contribution)
        self.ln4 = nn.Linear(tabular_contribution+img_contribution, n_classes) #20 perchè 10 derivano da img e 10 da tab
        

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.FloatTensor):
            zero_pads = zero_pads

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    conv1x1x1(self.in_planes, planes * block.expansion, stride),
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x, tab):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        tab = self.ln1(tab)
        tab = self.relu(tab)
        tab = self.ln2(tab)
        tab = self.relu(tab)
        tab = self.ln3(tab)
        tab = self.relu(tab)
        
        x = torch.cat((x, tab), dim=1)
        x= self.relu(x)
        
        x = self.ln4(x)        

        return x


def ResNet18(in_channels, num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_channels=in_channels, num_classes=num_classes)


def generate_model(model_depth, **kwargs):
    assert model_depth in [10, 18, 34, 50, 101, 152, 200]

    if model_depth == 10:
        model = ResNet(BasicBlock, [1, 1, 1, 1], get_inplanes(), **kwargs)
    elif model_depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], get_inplanes(), **kwargs)
    elif model_depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], get_inplanes(), **kwargs)
    elif model_depth == 152:
        model = ResNet(Bottleneck, [3, 8, 36, 3], get_inplanes(), **kwargs)
    elif model_depth == 200:
        model = ResNet(Bottleneck, [3, 24, 36, 3], get_inplanes(), **kwargs)

    return model


model = generate_model(18, n_input_channels=1, widen_factor=1.0, 
                       n_classes=1, img_contribution=10, tabular_val=len(test_data[i][1]), tabular_contribution=10)
model = model.double()
print(model)
#model = ResNet18(in_channels=91, num_classes=2)
#model = Net(in_channels=n_channels, num_classes=n_classes)
#model = model_to_syncbn(Conv3dConverter(model, i3d_repeat_axis=None))

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu

In [12]:
#img_x, label_x = tv_set[0]
#img_x.shape

#img_x = img_x.unsqueeze(dim=0)
#img_x = img_x.unsqueeze(dim=0)
#output = model(img_x)
#output.shape

#output = model(img_x)
#output.shape

In [93]:
def train(net, loaders, optimizer, criterion, epochs=500, dev=dev, save_param = False, model_name="adni_multi_input"):
    torch.manual_seed(myseed)
    start_time = time.time()
    try:
        net = net.to(dev)
        #print(net)
        # Initialize history
        history_loss = {"train": [], "val": [], "test": []}
        history_accuracy = {"train": [], "val": [], "test": []}
        # Store the best val accuracy
        best_val_accuracy = 0

        # Process each epoch
        for epoch in range(epochs):
            # Initialize epoch variables
            sum_loss = {"train": 0, "val": 0, "test": 0}
            sum_accuracy = {"train": 0, "val": 0, "test": 0}
            # Process each split
            for split in ["train", "val", "test"]:
                if split == "train":
                    net.train()
                else:
                    net.eval()
                # Process each batch
                for (image, tabular, labels) in loaders[split]:
                    # Move to CUDA
                    images = image.to(dev)
                    tabular = tabular.to(dev)
                    labels = labels.to(dev)
                    # Reset gradients
                    optimizer.zero_grad()
                    # Compute output
                    pred = net(image, tabular)
                    #pred = pred.squeeze(dim=1) # Output shape is [Batch size, 1], but we want [Batch size]
                    labels = labels.unsqueeze(1)
                    labels = labels.float()
                    loss = criterion(pred, labels)
                    # Update loss
                    sum_loss[split] += loss.item()
                    # Check parameter update
                    if split == "train":
                        # Compute gradients
                        loss.backward()
                        # Optimize
                        optimizer.step()
                    # Compute accuracy
                    #pred_labels = pred.argmax(1) + 1
                    pred_labels = (pred >= 0).long() # Binarize predictions to 0 and 1
                    batch_accuracy = (pred_labels == labels).sum().item()/image.size(0)
                    # Update accuracy
                    sum_accuracy[split] += batch_accuracy
                scheduler.step()
            # Compute epoch loss/accuracy
            epoch_loss = {split: sum_loss[split]/len(loaders[split]) for split in ["train", "val", "test"]}
            epoch_accuracy = {split: sum_accuracy[split]/len(loaders[split]) for split in ["train", "val", "test"]}

            # Store params at the best validation accuracy
            if save_param and epoch_accuracy["val"] > best_val_accuracy:
                #torch.save(net.state_dict(), f"{net.__class__.__name__}_best_val.pth")
                torch.save(net.state_dict(), f"{model_name}_best_val.pth")
                best_val_accuracy = epoch_accuracy["val"]

            # Update history
            for split in ["train", "val", "test"]:
                history_loss[split].append(epoch_loss[split])
                history_accuracy[split].append(epoch_accuracy[split])
                
            writer.add_scalar("Train Loss", epoch_loss['train'], epoch)
            writer.add_scalar("Valid Loss", epoch_loss['val'], epoch)
            writer.add_scalar("Test Loss", epoch_loss['test'], epoch)
            writer.add_scalar("Train Accuracy", epoch_accuracy['train'], epoch)
            writer.add_scalar("Valid Accuracy", epoch_accuracy['val'], epoch)
            writer.add_scalar("Test Accuracy", epoch_accuracy['test'], epoch)
            writer.add_scalar("ETA", time.time()-start_time, epoch)
            
            # Print info
            print(f"Epoch {epoch+1}:",
                  f"TrL={epoch_loss['train']:.4f},",
                  f"TrA={epoch_accuracy['train']:.4f},",
                  f"VL={epoch_loss['val']:.4f},",
                  f"VA={epoch_accuracy['val']:.4f},",
                  f"TeL={epoch_loss['test']:.4f},",
                  f"TeA={epoch_accuracy['test']:.4f},",
                  f"LR={optimizer.param_groups[0]['lr']:.5f},"
                  f"s={time.time()-start_time:.4f},")
    except KeyboardInterrupt:
        print("Interrupted")
    finally:
        # Plot loss
        plt.title("Loss")
        for split in ["train", "val", "test"]:
            plt.plot(history_loss[split], label=split)
        plt.legend()
        plt.show()
        # Plot accuracy
        plt.title("Accuracy")
        for split in ["train", "val", "test"]:
            plt.plot(history_accuracy[split], label=split)
        plt.legend()
        plt.show()

In [94]:
def reset_weights(m):

    if isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
        m.reset_parameters()


In [ ]:
experiment_name="run1"
generator = torch.Generator()
generator.manual_seed(myseed)
test_loader = DataLoader(test_data,  batch_size=8, num_workers=0, drop_last=False, shuffle=False, generator=generator)

epochs = 200

tv_labels = adni_tv['labels'].tolist()

skf = StratifiedKFold(n_splits=5)
for fold,(train_idx,val_idx) in enumerate(skf.split(tv_data, tv_labels)):
    writer = SummaryWriter(f"runs/adni2_multi_input/{experiment_name}/{fold}", filename_suffix=f"_F{fold}_E{epochs}")
    print('------------fold no---------{}----------------------'.format(fold))   
    train_df = adni_tv.iloc[train_idx]
    train_set = ImgDataset(adni_df=train_df)

    val_df = adni_tv.iloc[val_idx]
    val_set = ImgDataset(adni_df=val_df)
    
    train_loader = DataLoader(train_set, batch_size=8, drop_last=False)
    val_loader = DataLoader(val_set, batch_size=8, drop_last=False)
    
    # Define dictionary of loaders
    loaders = {"train": train_loader,
               "val": val_loader,
               "test": test_loader}

    # Model Params
    optimizer = optim.Adam(model.parameters(), lr = 0.0001)
    # Define a loss 
    criterion = nn.BCEWithLogitsLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.01, epochs=epochs, steps_per_epoch=len(train_loader))
    
    
    #model, optimizer = ipex.optimize(model, optimizer=optimizer,dtype=torch.double)
    #model = model.float()
    # Train model
    train(model, loaders, optimizer, criterion, epochs=epochs, dev=dev)
    writer.flush()
    writer.close()
    model.apply(reset_weights)

------------fold no---------0----------------------


In [30]:
# Check the tensorboard aftern enabling the port fwd using same port: localhost:6002
!tensorboard --logdir runs/adni2_img-only --bind_all --load_fast=false --port=6002

TensorFlow installation not found - running with reduced feature set.
TensorBoard 2.11.0 at http://aiicxnode2:6002/ (Press CTRL+C to quit)
^C
